In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
train = pd.read_csv("../data/train-test-split/TrainSet.csv", sep=",")
validation = pd.read_csv("../data/train-test-split/ValidationSet.csv", sep=",")
test = pd.read_csv("../data/train-test-split/TestSet.csv", sep=",")

In [ ]:
# Global variables

NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

THRESHOLD = 0.4   # For feature selection

In [ ]:
# Get the independent variables

columns_to_predict = ["kg_CO2/kWh", "Avg solar generation"]

independent_variables = []

for column in train:
    if abs(train[column].corr(train[columns_to_predict[0]])) > THRESHOLD:
        independent_variables.append(column)

independent_variables = [var for var in independent_variables if var not in columns_to_predict]
    
if "Index" in independent_variables:
    independent_variables.remove("Index")
if "Solar Generation (W/kW)_1" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_1")
if "Solar Generation (W/kW)_2" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_2")
if "Solar Generation (W/kW)_3" in independent_variables:
    independent_variables.remove("Solar Generation (W/kW)_3")
if "Hour_2" in independent_variables:
    independent_variables.remove("Hour_2")
if "Hour_3" in independent_variables:
    independent_variables.remove("Hour_3")
    
print(independent_variables)

In [ ]:
# Split the X and Y for all sets

# Train set
X_train_default = train[independent_variables]
Y_train_default = train[columns_to_predict]

# Validation set, also include the data from train that was used only as output to get more datapoints
X_val_default = pd.concat([X_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[independent_variables]], ignore_index=True)
Y_val_default = pd.concat([Y_train_default.tail(NUM_OF_TIMESTEPS_OUTPUT), validation[columns_to_predict]], ignore_index=True)

# Test set, also include the data from train that was used only as output to get more datapoints
X_test_default = pd.concat([X_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[independent_variables]], ignore_index=True)
Y_test_default = pd.concat([Y_val_default.tail(NUM_OF_TIMESTEPS_OUTPUT), test[columns_to_predict]], ignore_index=True)

NUM_OF_ROWS_TRAIN, NUM_OF_FEATURES = X_train_default.shape

print(X_train_default.shape)
print(X_val_default.shape)
print(X_test_default.shape)

In [ ]:
# Function to prepare the data into batches that will be passed into the model

def create_sequences(input_data, output_data, timesteps_input, timesteps_output):
    sequences, targets = [], []
    for i in range(len(input_data) - timesteps_input - timesteps_output + 1):
        seq = input_data[i:i + timesteps_input]
        target = output_data[i + timesteps_input: i + timesteps_input + timesteps_output]
        sequences.append(seq)
        targets.append(target)

    return np.array(sequences), np.array(targets)

In [ ]:
X_train, Y_train = create_sequences(X_train_default, Y_train_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_val, Y_val = create_sequences(X_val_default, Y_val_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
X_test, Y_test = create_sequences(X_test_default, Y_test_default, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)

print(f"X_train = {X_train.shape}, Y_train = {Y_train.shape}\n"
      f"X_val = {X_val.shape}, Y_val = {Y_val.shape}\n"
      f"X_test = {X_test.shape}, Y_test = {Y_test.shape}")

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras import layers, models
import time

tscv = TimeSeriesSplit(n_splits = 4, test_size=24) # For cross-validation

callback = keras.callbacks.EarlyStopping(monitor='loss', patience = 4)

# keep track of the losses
loss = []
val_loss = []
loss_1 = []
val_loss_1 = []
loss_2 = []
val_loss_2 = []

# Input layer
input_layer = layers.Input(shape=(NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))

# GRU layers for variable 1
gru1 = layers.GRU(48, activation='leaky_relu', return_sequences=True)(input_layer)
gru2 = layers.GRU(48, activation='leaky_relu', return_sequences=True)(gru1)
gru3 = layers.GRU(24, activation='leaky_relu', return_sequences=False)(gru2)
output_variable1 = layers.Dense(24, name='output_variable1')(gru3)

# GRU layers for variable 2
gru4 = layers.GRU(48, activation='leaky_relu', return_sequences=True)(input_layer)
gru5 = layers.GRU(48, activation='leaky_relu', return_sequences=True)(gru4)
gru6 = layers.GRU(24, activation='leaky_relu', return_sequences=False)(gru5)
output_variable2 = layers.Dense(24, name='output_variable2')(gru6)


# Define the model
model = models.Model(inputs=input_layer, outputs=[output_variable1, output_variable2])

model.compile(
    optimizer='adam',
    loss={'output_variable1': 'mean_squared_error', 'output_variable2': 'mean_squared_error'}
)

# keep track of the training time      
start = time.time()

for train_data, test_data in tscv.split(X_train):
    X_train_current_split, X_test_current_split = X_train[train_data], X_train[test_data]
    y_train_current_split, y_test_current_split = Y_train[train_data], Y_train[test_data]
    
    history = model.fit(X_train_current_split, y={"output_variable1": y_train_current_split[:, :, 0], 
                                                  "output_variable2": y_train_current_split[:, :, 1]},
              epochs=50, 
              validation_data=(
                  X_test_current_split,
                {
                    "output_variable1": y_test_current_split[:, :, 0],
                    "output_variable2": y_test_current_split[:, :, 1],
                },
              ),
              verbose=1,
              callbacks=callback
    )
    loss.append(model.history.history['loss'])
    val_loss.append(model.history.history['val_loss'])
    
    loss_1.append(model.history.history['output_variable1_loss'])
    val_loss_1.append(model.history.history['val_output_variable1_loss'])
    
    loss_2.append(model.history.history['output_variable2_loss'])
    val_loss_2.append(model.history.history['val_output_variable2_loss'])
    
end = time.time()

print(f"Training time = {end - start} seconds")

In [ ]:
plt.figure(figsize=(12, 12))

for i, column in enumerate(loss):
    plt.subplot(2, 2, i+1)
    plt.plot(column, label="Train error")
    plt.plot(val_loss[i], label="Validation error")
    plt.xlabel('Epoch')
    plt.ylabel('Validation loss (MSE)')
    plt.legend()
    plt.title(f'Validation loss for Fold {i+1}')

plt.suptitle('Total Validation Loss across folds in GRU', fontsize=18, y=1)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))

for i, column in enumerate(loss_1):
    plt.subplot(2, 2, i+1)
    plt.plot(column, label="Train error")
    plt.plot(val_loss_1[i], label="Validation error")
    plt.xlabel('Epoch')
    plt.ylabel('Validation loss (MSE)')
    plt.legend()
    plt.title(f'Validation loss for Fold {i+1}')

plt.suptitle('Validation Loss across folds for Carbon Intensity in GRU', fontsize=18, y=1)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))

for i, column in enumerate(loss_2):
    plt.subplot(2, 2, i+1)
    plt.plot(column, label="Train error")
    plt.plot(val_loss_2[i], label="Validation error")
    plt.xlabel('Epoch')
    plt.ylabel('Validation loss (MSE)')
    plt.legend()
    plt.title(f'Validation loss for Fold {i+1}')

plt.suptitle('Validation Loss across folds for Solar Generation in GRU', fontsize=18, y=1)
plt.tight_layout()
plt.show()

In [ ]:
# Export the loss

headers = []
loss_df = []

for i, column in enumerate(loss):
    loss_df.append(column)
    loss_df.append(val_loss[i])
    headers.append(f"Fold {i} Train")
    headers.append(f"Fold {i} Val")
    

loss_df = pd.DataFrame(loss_df).T
loss_df.to_csv('../data/results/loss_gru.csv', index=False, header=headers)

In [ ]:
# Evaluate the model on the test data
predictions_1 = []
predictions_2 = []

for i in range(len(Y_val)):
    current_batch = X_val[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    curr_pred1, curr_pred2 = model.predict(current_batch)
    predictions_1.append(curr_pred1)
    predictions_2.append(curr_pred2)

In [ ]:
# Plot the evaluation predictions for CI

plt.figure(figsize=(20, 48))

i = 1

for num, column in enumerate(predictions_1):
    plt.subplot(9, 3, i)
    i += 1
    plt.plot(column[0], label="Prediction")
    plt.plot(Y_val[num, :, 0], label="Actual")
    plt.xlabel('Timestamp', fontsize=20)
    plt.ylabel('Carbon Intensity (kg/kWh)', fontsize=20)
    plt.legend(fontsize=14)
    plt.title(f'Prediction number {num+1}', fontsize=22)
    plt.tick_params(labelsize=16)

plt.suptitle('Predicted and actual Solar Generation', fontsize=26, y=1)
plt.tight_layout()
plt.show()

In [ ]:
# Plot the evaluation predictions for SG

plt.figure(figsize=(20, 48))

i = 1

for num, column in enumerate(predictions_2):
    plt.subplot(9, 3, i)
    i += 1
    plt.plot(column[0], label="Prediction")
    plt.plot(Y_val[num, :, 1], label="Actual")
    plt.xlabel('Timestamp', fontsize=20)
    plt.ylabel('Carbon Intensity (kg/kWh)', fontsize=20)
    plt.legend(fontsize=14)
    plt.title(f'Prediction number {num+1}', fontsize=22)
    plt.tick_params(labelsize=16)

plt.suptitle('Predicted and actual Solar Generation', fontsize=26, y=1)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate the RMSE

rmse = 0
for i in range(len(predictions_1)):
    rmse += sqrt(mean_squared_error(predictions_1[i][0], Y_val[i, :, 0]))

rmse /= len(predictions_1)    
print(f"RMSE for {columns_to_predict[0]} = {rmse}")

rmse = 0
for i in range(len(predictions_2)):
    rmse += sqrt(mean_squared_error(predictions_2[i][0], Y_val[i, :, 1]))

rmse /= len(predictions_2)    
print(f"RMSE for {columns_to_predict[1]} = {rmse}")

In [ ]:
# Evaluate the model on the test data
predictions_1_test = []
predictions_2_test = []

total_time = 0

for i in range(len(Y_test)):
    current_batch = X_test[i].reshape((1, NUM_OF_TIMESTEPS_INPUT, NUM_OF_FEATURES))
    start = time.time()
    curr_pred1, curr_pred2 = model.predict(current_batch)
    end = time.time()
    total_time += end-start
    predictions_1_test.append(curr_pred1)
    predictions_2_test.append(curr_pred2)
    
print(f"Average prediction time is {total_time/len(Y_test)} seconds")

In [ ]:
# Plot the test predictions for CI

plt.figure(figsize=(20, 48))

i = 1

for num, column in enumerate(predictions_1_test):
    plt.subplot(9, 3, i)
    i += 1
    plt.plot(column[0], label="Prediction")
    plt.plot(Y_test[num, :, 0], label="Actual")
    plt.xlabel('Timestamp', fontsize=20)
    plt.ylabel('Carbon Intensity (kg/kWh)', fontsize=20)
    plt.legend(fontsize=14)
    plt.title(f'Prediction number {num+1}', fontsize=22)
    plt.tick_params(labelsize=16)

plt.suptitle('Predicted and actual Carbon Intensity', fontsize=26, y=1)
plt.tight_layout()
plt.show()

In [ ]:
# Plot the test predictions for SG

plt.figure(figsize=(20, 48))

i = 1

for num, column in enumerate(predictions_2_test):
    plt.subplot(9, 3, i)
    i += 1
    plt.plot(column[0], label="Prediction")
    plt.plot(Y_test[num, :, 1], label="Actual")
    plt.xlabel('Timestamp', fontsize=20)
    plt.ylabel('Carbon Intensity (kg/kWh)', fontsize=20)
    plt.legend(fontsize=14)
    plt.title(f'Prediction number {num+1}', fontsize=22)
    plt.tick_params(labelsize=16)

plt.suptitle('Predicted and actual Solar Generation', fontsize=26, y=1)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate the RMSE for the test predictions

rmse = 0
for i in range(len(predictions_1_test)):
    rmse += sqrt(mean_squared_error(predictions_1_test[i][0], Y_test[i, :, 0]))

rmse /= len(predictions_1_test)    
print(f"RMSE for {columns_to_predict[0]} = {rmse}")

rmse = 0
for i in range(len(predictions_2_test)):
    rmse += sqrt(mean_squared_error(predictions_2_test[i][0], Y_test[i, :, 1]))

rmse /= len(predictions_2_test)    
print(f"RMSE for {columns_to_predict[1]} = {rmse}")

In [ ]:
# Export the predictions

a = []
for column in predictions_1_test:
    a.append(column[0])
    
b = []
for column in predictions_2_test:
    b.append(column[0])

predictions1 = pd.DataFrame(a).T
predictions1.to_csv('../data/results/carbon_gru.csv', index=False, header=False)
predictions2 = pd.DataFrame(b).T
predictions2.to_csv('../data/results/solar_gru.csv', index=False, header=False)